<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=184103482" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=183010840" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182786979" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182712552" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182257992" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Details of Steps can be found on Google Slides via:¶

https://docs.google.com/presentation/d/1sb3QkXiYooHqi3p-tkGVUqwqFKd-601_pzU96W1drw0/edit?usp=sharing

In [1]:
from kaggle_secrets import UserSecretsClient
wandb_key_label = "WANDB_KEY"
wandb_key= UserSecretsClient().get_secret(wandb_key_label)

In [2]:
import git
git.Repo.clone_from('https://github.com/Lumin-Lab/BerCuration', '/kaggle/working/scarf')

<git.repo.base.Repo '/kaggle/working/scarf/.git'>

In [ ]:
!pip install -r /kaggle/working/scarf/requirements.txt

In [3]:
import os
def save_csv_file(df, path):
    dir_name = os.path.dirname(path)
    if dir_name:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    df.to_csv(path, index=False)

In [4]:
raw_data_path = "/kaggle/input/ber-stratified-samples/BER_stratified_sample.csv"
n_splits = 5
output_dir = "/kaggle/working/output"
config_dir="/kaggle/working/scarf/configs"
scarf_model_name = "scarf"
mlp_model_name = "mlp"
scarf_batch_size = 32
scarf_epochs = 15
scarf_lr = 3e-5
scarf_emb_dim = 32
scarf_encoder_depth = 3
scarf_corruption_rate=0.3
max_depth = 5

In [5]:
import pandas as pd
X_raw = pd.read_csv(raw_data_path)

/tmp/ipykernel_33/797606596.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  X_raw = pd.read_csv(raw_data_path)


In [6]:
from sklearn.model_selection import KFold
import os
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X_raw)):
    save_csv_file(X_raw.iloc[train_index], f"{output_dir}/split_{i+1}/raw_train.csv")
    save_csv_file(X_raw.iloc[test_index], f"{output_dir}/split_{i+1}/raw_test.csv")

In [7]:
"""input: data_path={raw_data_path}" ,
output:{output_dir}/processed.csv """

command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}" \
      --data_path={raw_data_path} \
      --output_csv_name "processed" \
      --is_train
    """
os.system(command)

0

In [8]:
files_to_copy = ["column_type_classification.yaml", "encoder.joblib", "scaler.joblib", "train_stats.json"]
for i in range(n_splits):
    for file_name in files_to_copy:
        src = f"{output_dir}/{file_name}"
        dest = f"{output_dir}/split_{i+1}/{file_name}"
        os.system(f"cp {src} {dest}")

In [9]:
for i in range(n_splits):
    """input: data_path "{output_dir}/split_{i+1}/raw_train.csv" ,
    output:{output_dir}/split_{i+1}/processed_train.csv """
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "{config_dir}" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_train.csv" \
      --output_csv_name "processed_train" \
    """
    os.system(command)
    """input: data_path "{output_dir}/split_{i+1}/raw_test.csv" ,
    output:{output_dir}/split_{i+1}/processed_test.csv """
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "{config_dir}" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_test.csv" \
      --output_csv_name "processed_test"
    """
    os.system(command)

**Train the SCARF Encoder on the Train Dataset**

In [10]:
import wandb
wandb.login(key=wandb_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
import os
"""
Input: train_data_path="{output_dir}/split_{i+1}/processed_train.csv"
 
Output: /kaggle/working/output/split_{split}/scarf.pt  
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/run_scarf.py\
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --train_data_path="{output_dir}/split_{i+1}/processed_train.csv"\
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name="{scarf_model_name}" \
      --corruption_rate={scarf_corruption_rate} \
      --wandb_project_name='SCARF_Project' \
      --wandb_entity='urbancomp' 
    """

    os.system(command)

wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240617_214305-h5e201tm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SCARF_Project
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf/runs/h5e201tm


Epoch [1/15] - Train Loss: 3.010
Model saved at /kaggle/working/output/split_1/scarf.pt
Epoch [2/15] - Train Loss: 2.926
Model saved at /kaggle/working/output/split_1/scarf.pt
Epoch [3/15] - Train Loss: 2.866
Model saved at /kaggle/working/output/split_1/scarf.pt
Epoch [4/15] - Train Loss: 2.957
Epoch [5/15] - Train Loss: 2.884
Epoch [6/15] - Train Loss: 2.919
Epoch [7/15] - Train Loss: 2.896
Epoch [8/15] - Train Loss: 2.849
Model saved at /kaggle/working/output/split_1/scarf.pt
Epoch [9/15] - Train Loss: 2.942
Epoch [10/15] - Train Loss: 2.856
Epoch [11/15] - Train Loss: 2.905
Epoch [12/15] - Train Loss: 2.846
Model saved at /kaggle/working/output/split_1/scarf.pt
Epoch [13/15] - Train Loss: 2.819
Model saved at /kaggle/working/output/split_1/scarf.pt
Epoch [14/15] - Train Loss: 2.837
Epoch [15/15] - Train Loss: 2.808
Model saved at /kaggle/working/output/split_1/scarf.pt


wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240617_214858-44iyo9o4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SCARF_Project
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf/runs/44iyo9o4


Epoch [1/15] - Train Loss: 2.965
Model saved at /kaggle/working/output/split_2/scarf.pt
Epoch [2/15] - Train Loss: 2.877
Model saved at /kaggle/working/output/split_2/scarf.pt
Epoch [3/15] - Train Loss: 2.858
Model saved at /kaggle/working/output/split_2/scarf.pt
Epoch [4/15] - Train Loss: 2.856
Model saved at /kaggle/working/output/split_2/scarf.pt
Epoch [5/15] - Train Loss: 2.846
Model saved at /kaggle/working/output/split_2/scarf.pt
Epoch [6/15] - Train Loss: 2.843
Model saved at /kaggle/working/output/split_2/scarf.pt
Epoch [7/15] - Train Loss: 2.855
Epoch [8/15] - Train Loss: 2.918
Epoch [9/15] - Train Loss: 2.777
Model saved at /kaggle/working/output/split_2/scarf.pt
Epoch [10/15] - Train Loss: 2.823
Epoch [11/15] - Train Loss: 2.838
Epoch [12/15] - Train Loss: 2.910
Epoch [13/15] - Train Loss: 2.870
Epoch [14/15] - Train Loss: 2.816
Epoch [15/15] - Train Loss: 2.763
Model saved at /kaggle/working/output/split_2/scarf.pt


wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240617_215452-ysf9zjn1
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SCARF_Project
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf/runs/ysf9zjn1


Epoch [1/15] - Train Loss: 3.031
Model saved at /kaggle/working/output/split_3/scarf.pt
Epoch [2/15] - Train Loss: 2.878
Model saved at /kaggle/working/output/split_3/scarf.pt
Epoch [3/15] - Train Loss: 2.909
Epoch [4/15] - Train Loss: 2.838
Model saved at /kaggle/working/output/split_3/scarf.pt
Epoch [5/15] - Train Loss: 2.842
Epoch [6/15] - Train Loss: 2.988
Epoch [7/15] - Train Loss: 2.850
Epoch [8/15] - Train Loss: 2.856
Epoch [9/15] - Train Loss: 2.875
Epoch [10/15] - Train Loss: 2.844
Epoch [11/15] - Train Loss: 2.935
Epoch [12/15] - Train Loss: 2.755
Model saved at /kaggle/working/output/split_3/scarf.pt
Epoch [13/15] - Train Loss: 2.852
Epoch [14/15] - Train Loss: 2.893
Epoch [15/15] - Train Loss: 2.879


wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240617_220045-4abraezd
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SCARF_Project
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf/runs/4abraezd


Epoch [1/15] - Train Loss: 2.844
Model saved at /kaggle/working/output/split_4/scarf.pt
Epoch [2/15] - Train Loss: 2.919
Epoch [3/15] - Train Loss: 2.855
Epoch [4/15] - Train Loss: 2.836
Model saved at /kaggle/working/output/split_4/scarf.pt
Epoch [5/15] - Train Loss: 2.852
Epoch [6/15] - Train Loss: 2.901
Epoch [7/15] - Train Loss: 2.814
Model saved at /kaggle/working/output/split_4/scarf.pt
Epoch [8/15] - Train Loss: 2.811
Model saved at /kaggle/working/output/split_4/scarf.pt
Epoch [9/15] - Train Loss: 2.866
Epoch [10/15] - Train Loss: 2.949
Epoch [11/15] - Train Loss: 2.870
Epoch [12/15] - Train Loss: 2.884
Epoch [13/15] - Train Loss: 2.780
Model saved at /kaggle/working/output/split_4/scarf.pt
Epoch [14/15] - Train Loss: 2.934
Epoch [15/15] - Train Loss: 2.871


wandb: Currently logged in as: dan-liu. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: dan-liu (urbancomp). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240617_220642-f5r1qugx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SCARF_Project
wandb: ⭐️ View project at https://wandb.ai/urbancomp/Scarf
wandb: 🚀 View run at https://wandb.ai/urbancomp/Scarf/runs/f5r1qugx


Epoch [1/15] - Train Loss: 2.968
Model saved at /kaggle/working/output/split_5/scarf.pt
Epoch [2/15] - Train Loss: 3.054
Epoch [3/15] - Train Loss: 2.909
Model saved at /kaggle/working/output/split_5/scarf.pt
Epoch [4/15] - Train Loss: 2.907
Model saved at /kaggle/working/output/split_5/scarf.pt
Epoch [5/15] - Train Loss: 2.858
Model saved at /kaggle/working/output/split_5/scarf.pt
Epoch [6/15] - Train Loss: 2.965
Epoch [7/15] - Train Loss: 2.922
Epoch [8/15] - Train Loss: 2.949
Epoch [9/15] - Train Loss: 3.068
Epoch [10/15] - Train Loss: 2.937
Epoch [11/15] - Train Loss: 2.923
Epoch [12/15] - Train Loss: 2.964
Epoch [13/15] - Train Loss: 2.840
Model saved at /kaggle/working/output/split_5/scarf.pt
Epoch [14/15] - Train Loss: 2.916
Epoch [15/15] - Train Loss: 2.916


**Obtain the SCARF embeddings for different train splits, and save the result**

In [12]:
"""
Input: data_path="{output_dir}/split_{i+1}/processed_train.csv"
 
Output: {output_dir}/split_{i+1}/train.npy
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --data_path="{output_dir}/split_{i+1}/processed_train.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="train"
    """

    os.system(command)

Model loaded from /kaggle/working/output/split_1/scarf.pt
Model loaded from /kaggle/working/output/split_2/scarf.pt
Model loaded from /kaggle/working/output/split_3/scarf.pt
Model loaded from /kaggle/working/output/split_4/scarf.pt
Model loaded from /kaggle/working/output/split_5/scarf.pt


**Obtain the SCARF embeddings for different test splits, and save the result**

In [13]:
"""
Input: data_path="{output_dir}/split_{i+1}/processed_test.csv"
 
Output: {output_dir}/split_{i+1}/test.npy
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py\
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --data_path="{output_dir}/split_{i+1}/processed_test.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="test"
    """

    os.system(command)

Model loaded from /kaggle/working/output/split_1/scarf.pt
Model loaded from /kaggle/working/output/split_2/scarf.pt
Model loaded from /kaggle/working/output/split_3/scarf.pt
Model loaded from /kaggle/working/output/split_4/scarf.pt
Model loaded from /kaggle/working/output/split_5/scarf.pt


In [15]:
"""
Input: data_path="{output_dir}/processedt.csv"
 
Output: {output_dir}/split_1/all_emb.npy
"""

command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_1" \
      --data_path="{output_dir}/processed.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="all_emb"
    """

os.system(command)

Model loaded from /kaggle/working/output/split_1/scarf.pt


0

**Random Forest**

In [16]:
from sklearn.ensemble import RandomForestClassifier
def get_rf(max_depth=max_depth):
    return RandomForestClassifier(max_depth=max_depth, random_state=42)

**Original Data**

In [17]:
import numpy as np
processed = pd.read_csv(f"{output_dir}/processed.csv")
all_emb = np.load(f"{output_dir}/split_1/all_emb.npy")

In [20]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import cross_val_predict
from cleanlab import Datalab
from sklearn.metrics import f1_score

In [21]:
X_processed = processed.drop(columns=["EnergyRating"])
labels = processed['EnergyRating']
clf = get_rf()
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=kf,
    method="predict_proba",
)
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_processed.values)

knn_graph = KNN.kneighbors_graph(mode="distance")
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)

display(lab.report())


Finding label issues ...


2024-06-17 22:21:30.586010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 22:21:30.586153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 22:21:30.724107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Finding outlier issues ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Finding class_imbalance issues ...
Finding underperforming_group issues ...

Audit complete. 67266 issues found in the dataset.
Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
         label       56185
near_duplicate        8785
       outlier        2296

Dataset Information: num_examples: 121184, num_classes: 15


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 56185
Overall dataset quality in terms of this issue: 0.5532

Examples representing most severe instances of this issue:
       is_label_issue  label_score  given_label  predicted_label
24816            True     0.003259            0                7
12524            True     0.00

None

In [22]:
# Label Issues
label_issues_num =  56185
outlier_nums = 2296
duplicates_nums = 8785

issue_results = lab.get_issues("label")
sorted_issues = issue_results.sort_values("label_score").index

sorted_issues_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_issues, "Unnamed: 0"]
})
save_csv_file(sorted_issues_df[:label_issues_num], f"{output_dir}/random_forest/data/label_issues.csv")
print(len(sorted_issues_df))

print("Label Issues")
display(X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head())

# Outliers
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index
print("Outliers")
display(X_raw.iloc[sorted_outliers].head())

sorted_outliers_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_outliers[:outlier_nums], "Unnamed: 0"]
})
print(len(sorted_outliers_df))
save_csv_file(sorted_outliers_df, f"{output_dir}/random_forest/data/outliers.csv")


# Near-duplicate issues
duplicate_results = lab.get_issues("near_duplicate")
sorted_duplicates = duplicate_results.sort_values("near_duplicate_score").index
print("Near-duplicate issues")
display(duplicate_results.sort_values("near_duplicate_score").head())
sorted_duplicates_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_duplicates, "Unnamed: 0"]
})
print(len(sorted_duplicates_df))
save_csv_file(sorted_duplicates_df[:duplicates_nums], f"{output_dir}/random_forest/data/duplicates.csv")


121184
Label Issues


,Unnamed: 0,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,...,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC,given_label,predicted_label
24816,752940,Co. Galway,Semi-detached house,2001,Existing,A1,-21.61,65.640,0.55,0.13,...,067057011,0.050000,0.050000,1.143,12712.3,15487.50,0.507,-0.108,0,7
12524,117187,Co. Donegal,Detached house,2009,Final,A1,3.89,173.570,0.13,0.11,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.188,0.022,0,2
47324,648381,Dublin 11,Mid-terrace house,1976,Existing,A2,46.82,87.240,0.42,0.17,...,268030005,0.000222,0.012347,0.692,10529.0,8716.96,0.539,0.293,1,6
67446,644247,Co. Carlow,Semi-detached house,1987,Existing,A2,33.48,70.000,0.37,0.20,...,17045001,0.050000,0.050000,0.847,10998.2,9948.62,0.618,0.180,1,6
71754,483281,Dublin 15,End of terrace house,2018,Existing,A1,15.54,126.567,0.27,0.14,...,267028004/02,0.000222,0.012347,0.852,12323.4,11348.50,0.062,0.106,0,5


Outliers


,Unnamed: 0,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,...,ThirdWallAgeBandId,ThirdWallTypeId,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
99966,912339,Co. Galway,Detached house,1905,Existing,G,842.25,88.56,1.99,1.44,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,654970,Co. Louth,Detached house,2013,Provisional,A2,48.08,984.23,0.16,0.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.335,0.360
54221,1137153,Dublin 18,Detached house,1944,Existing,E1,311.70,510.65,2.10,2.30,...,NaN,NaN,267090003,0.050000,0.050000,0.0,155887.0,0.0,1.834,1.794
94225,331,Co. Kildare,Detached house,1943,Existing,G,481.80,71.00,2.16,0.13,...,NaN,NaN,87068001,0.089325,0.089325,NaN,NaN,NaN,NaN,NaN
3329,128742,Co. Laois,Mid-terrace house,1960,Existing,G,989.35,138.82,2.10,2.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2296
Near-duplicate issues


,is_near_duplicate_issue,near_duplicate_score,near_duplicate_sets,distance_to_nearest_neighbor
12099,True,0.0,"[110211, 12058, 46803]",0.0
67498,True,0.0,[104454],0.0
61290,True,0.0,"[31822, 41350, 44527, 90228, 113967, 58970]",0.0
80049,True,0.0,[30037],0.0
80048,True,0.0,"[38004, 60227, 21344, 68517]",0.0


121184


**Before Removal**

In [23]:
results = []

for i in range(n_splits):
    clf = get_rf()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"])
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/random_forest/data/results_after_removal.csv")
    

Split: 1, Accuracy: 0.36885753187275655, F1 Score: 0.24045867539550603
Split: 2, Accuracy: 0.3634938317448529, F1 Score: 0.2362148571977429
Split: 3, Accuracy: 0.3649379048562116, F1 Score: 0.24136144933009357
Split: 4, Accuracy: 0.3606882039856418, F1 Score: 0.23683361864630365
Split: 5, Accuracy: 0.36165208780326785, F1 Score: 0.23467134284219557


After Removal

In [24]:
for i in range(n_splits):
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    label_issue_index = pd.read_csv(f"{output_dir}/random_forest/data/label_issues.csv")
    train_df_removed = train_df[~train_df["Unnamed: 0"].isin(label_issue_index["Unnamed: 0"])]
    print(len(train_df_removed), len(train_df_removed)/len(train_df))
    save_csv_file(train_df_removed, f"{output_dir}/split_{i+1}/train_removed.csv")

/tmp/ipykernel_33/4162368144.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


52027 0.5366540480881307


/tmp/ipykernel_33/4162368144.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


51995 0.5363239708294223


/tmp/ipykernel_33/4162368144.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


51910 0.5354472031109782


/tmp/ipykernel_33/4162368144.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


52035 0.5367365674028077


/tmp/ipykernel_33/4162368144.py:2: DtypeWarning: Columns (156,162) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


52029 0.5366691422205718


In [28]:
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir {config_dir} \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/train_removed.csv" \
      --output_csv_name "processed_train_removed" \
    """
    os.system(command)

In [29]:
results = []

for i in range(n_splits):
    clf = get_rf()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train_removed.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"])
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/random_forest/data/results_after_removal.csv")

Split: 1, Accuracy: 0.3130750505425589, F1 Score: 0.17655629563207229
Split: 2, Accuracy: 0.31340512439658375, F1 Score: 0.17825791855950388
Split: 3, Accuracy: 0.3128274951520403, F1 Score: 0.17703563003887263
Split: 4, Accuracy: 0.3123736436027561, F1 Score: 0.17732593041582576
Split: 5, Accuracy: 0.30673378445288, F1 Score: 0.17241522018628241


**Scarf Embedding**

In [30]:

X_processed = pd.DataFrame(all_emb, columns=[f"feature_{i+1}" for i in range(all_emb.shape[1])])
labels = processed['EnergyRating']
clf = get_rf()
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=kf,
    method="predict_proba",
)
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_processed.values)

knn_graph = KNN.kneighbors_graph(mode="distance")
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)
display(lab.report())

Finding label issues ...
Finding outlier issues ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Finding class_imbalance issues ...
Finding underperforming_group issues ...

Audit complete. 64678 issues found in the dataset.
Here is a summary of the different kinds of issues found in the data:

           issue_type  num_issues
                label       52926
       near_duplicate        7915
              outlier        3832
underperforming_group           5

Dataset Information: num_examples: 121184, num_classes: 15


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 52926
Overall dataset quality in terms of this issue: 0.4681

Examples representing most severe instances of this issue:
        is_label_issue  label_score  given_label  predicted_label
24816  

None

In [31]:
# Label Issues
label_issues_num =  52164
outlier_nums = 3752
duplicates_nums = 7983

issue_results = lab.get_issues("label")
sorted_issues = issue_results.sort_values("label_score").index

sorted_issues_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_issues, "Unnamed: 0"]
})
save_csv_file(sorted_issues_df[:label_issues_num], f"{output_dir}/random_forest/emb/label_issues.csv")
print(len(sorted_issues_df))

print("Label Issues")
display(X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head())

# Outliers
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index
print("Outliers")
display(X_raw.iloc[sorted_outliers].head())

sorted_outliers_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_outliers[:outlier_nums], "Unnamed: 0"]
})
print(len(sorted_outliers_df))
save_csv_file(sorted_outliers_df, f"{output_dir}/random_forest/emb/outliers.csv")


# Near-duplicate issues
duplicate_results = lab.get_issues("near_duplicate")
sorted_duplicates = duplicate_results.sort_values("near_duplicate_score").index
print("Near-duplicate issues")
display(duplicate_results.sort_values("near_duplicate_score").head())
sorted_duplicates_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_duplicates, "Unnamed: 0"]
})
print(len(sorted_duplicates_df))
save_csv_file(sorted_duplicates_df[:duplicates_nums], f"{output_dir}/random_forest/emb/duplicates.csv")


121184
Label Issues


,Unnamed: 0,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,...,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC,given_label,predicted_label
24816,752940,Co. Galway,Semi-detached house,2001,Existing,A1,-21.61,65.64,0.55,0.13,...,067057011,0.050000,0.050000,1.143,12712.30,15487.50,0.507,-0.108,0,7
111046,481391,Dublin 6,Mid-floor apartment,1900,Existing,A3,57.66,17.59,0.25,0.00,...,268132014,0.050000,0.050000,0.396,1558.85,1021.80,0.172,0.170,2,10
96431,31323,Co. Carlow,Top-floor apartment,2011,Existing,G,510.16,69.00,0.37,0.40,...,17051005,0.000222,0.012347,NaN,NaN,NaN,NaN,NaN,14,6
31972,484514,Co. Leitrim,Semi-detached house,1900,Existing,B1,84.89,126.82,0.31,0.13,...,117070001/117070002,0.050000,0.050000,0.467,18847.80,9429.72,0.751,0.515,3,9
104299,481398,Dublin 6,Top-floor apartment,1900,Existing,A3,57.17,19.13,0.25,0.11,...,268132014,0.050000,0.050000,0.397,1653.75,1091.02,0.169,0.169,2,10


Outliers


,Unnamed: 0,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,...,ThirdWallAgeBandId,ThirdWallTypeId,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
99966,912339,Co. Galway,Detached house,1905,Existing,G,842.25,88.56,1.99,1.44,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,654970,Co. Louth,Detached house,2013,Provisional,A2,48.08,984.23,0.16,0.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.335,0.360
110292,227162,Co. Wicklow,Detached house,1970,Existing,D2,262.02,860.31,2.05,2.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94225,331,Co. Kildare,Detached house,1943,Existing,G,481.80,71.00,2.16,0.13,...,NaN,NaN,87068001,0.089325,0.089325,NaN,NaN,NaN,NaN,NaN
54221,1137153,Dublin 18,Detached house,1944,Existing,E1,311.70,510.65,2.10,2.30,...,NaN,NaN,267090003,0.050000,0.050000,0.0,155887.0,0.0,1.834,1.794


3752
Near-duplicate issues


,is_near_duplicate_issue,near_duplicate_score,near_duplicate_sets,distance_to_nearest_neighbor
33156,True,0.0,[37576],0.0
104541,True,0.0,[56916],0.0
54683,True,0.0,"[74172, 67755]",0.0
25554,True,0.0,[85878],0.0
104553,True,0.0,"[11315, 53432]",0.0


121184


**Before Removal**

In [32]:
results = []

for i in range(n_splits):
    clf = get_rf()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}/train.npy")
    test_X = np.load(f"{output_dir}/split_{i+1}/test.npy")
    clf.fit(train_X, train_df["EnergyRating"])
    acc = clf.score(test_X, test_df["EnergyRating"],
             )
    predict = clf.predict(test_X)
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/random_forest/emb/results_before_removal.csv")

Split: 1, Accuracy: 0.38585633535503566, F1 Score: 0.2823943318526506
Split: 2, Accuracy: 0.380080042909601, F1 Score: 0.2815465427112159
Split: 3, Accuracy: 0.376531748978834, F1 Score: 0.27809605921699393
Split: 4, Accuracy: 0.3766142674423402, F1 Score: 0.2823134228941602
Split: 5, Accuracy: 0.3749381085987787, F1 Score: 0.2780300406332188


**After Removal**

In [33]:
original_rating_encoding = {
    "A1": 0,
    "A2": 1,
    "A3": 2,
    "B1": 3,
    "B2": 4,
    "B3": 5,
    "C1": 6,
    "C2": 7,
    "C3": 8,
    "D1": 9,
    "D2": 10,
    "E1": 11,
    "E2": 12,
    "F": 13,
    "G": 14
}

In [34]:
for i in range(n_splits):
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}/train.npy")
    
    label_issue_index = pd.read_csv(f"{output_dir}/random_forest/emb/label_issues.csv")
    train_df_removed = train_df[~train_df["Unnamed: 0"].isin(label_issue_index["Unnamed: 0"])]
    train_X_removed = train_X[train_df_removed.index]
    train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
    train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].map(original_rating_encoding)
    print(len(train_df_removed), len(train_df_removed)/len(train_df))
    save_csv_file(train_df_removed, f"{output_dir}/split_{i+1}/train_removed.csv")
    np.save(f"{output_dir}/split_{i+1}/train_removed.npy", train_X_removed)

/tmp/ipykernel_33/2867342498.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
/tmp/ipykernel_33/2867342498.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
/tmp/ipykernel_33/2867342498.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['Ener

55107 0.5684239842388109


/tmp/ipykernel_33/2867342498.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
/tmp/ipykernel_33/2867342498.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
/tmp/ipykernel_33/2867342498.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['Ener

55311 0.5705282267630767


/tmp/ipykernel_33/2867342498.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
/tmp/ipykernel_33/2867342498.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
/tmp/ipykernel_33/2867342498.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['Ener

55129 0.5686509123541729


/tmp/ipykernel_33/2867342498.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
/tmp/ipykernel_33/2867342498.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
/tmp/ipykernel_33/2867342498.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['Ener

55312 0.5705385416774114


/tmp/ipykernel_33/2867342498.py:2: DtypeWarning: Columns (156,162) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
/tmp/ipykernel_33/2867342498.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
/tmp/ipykernel_33/2867342498.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_removed['EnergyRating'] = train_df_removed['EnergyRa

55221 0.5695940091595495


In [35]:
results = []

for i in range(n_splits):
    clf = get_rf()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}//train_removed.npy")
    test_X = np.load(f"{output_dir}/split_{i+1}/test.npy")
    clf.fit(train_X, train_df["EnergyRating"])
    acc = clf.score(test_X, test_df["EnergyRating"],
             )
    predict = clf.predict(test_X)
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/random_forest/emb/results_after_removal.csv")



/tmp/ipykernel_33/3560689056.py:5: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")


Split: 1, Accuracy: 0.3750876758674753, F1 Score: 0.2635707249000939


/tmp/ipykernel_33/3560689056.py:5: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")


Split: 2, Accuracy: 0.37269464042579525, F1 Score: 0.26066285176993925


/tmp/ipykernel_33/3560689056.py:5: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")


Split: 3, Accuracy: 0.3676610141519165, F1 Score: 0.25674041769816314


/tmp/ipykernel_33/3560689056.py:5: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")


Split: 4, Accuracy: 0.36914634649502825, F1 Score: 0.260345964678991


/tmp/ipykernel_33/3560689056.py:5: DtypeWarning: Columns (156,162) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")


Split: 5, Accuracy: 0.36540683281069486, F1 Score: 0.25745068711606794


**MLP**

In [36]:
from sklearn.neural_network import MLPClassifier
hidden_layer_sizes = [256 , 128 , 64 , 32 , 16]
max_iter = 20
batch_size = 32
def get_mlp(hidden_layer_sizes=hidden_layer_sizes,
           max_iter= max_iter,
           batch_size = batch_size):
    return MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                        random_state= 42,
                        max_iter= max_iter,
                        batch_size = batch_size)

In [37]:
X_processed = processed.drop(columns=["EnergyRating"])
labels = processed['EnergyRating']
clf = get_mlp()
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=kf,
    method="predict_proba",
)
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_processed.values)

knn_graph = KNN.kneighbors_graph(mode="distance")
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)

display(lab.report())

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py

Finding label issues ...
Finding outlier issues ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Finding class_imbalance issues ...
Finding underperforming_group issues ...

Audit complete. 46959 issues found in the dataset.
Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
         label       35878
near_duplicate        8785
       outlier        2296

Dataset Information: num_examples: 121184, num_classes: 15


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 35878
Overall dataset quality in terms of this issue: 0.7465

Examples representing most severe instances of this issue:
        is_label_issue   label_score  given_label  predicted_label
56543             True  1.548042e-13            2                1
1

None

**Original Data**

In [38]:
# Label Issues
label_issues_num =  36939
outlier_nums = 2296
duplicates_nums = 8785

issue_results = lab.get_issues("label")
sorted_issues = issue_results.sort_values("label_score").index

sorted_issues_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_issues, "Unnamed: 0"]
})
save_csv_file(sorted_issues_df[:label_issues_num], f"{output_dir}/mlp/data/label_issues.csv")
print(len(sorted_issues_df))

print("Label Issues")
display(X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head())

# Outliers
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index
print("Outliers")
display(X_raw.iloc[sorted_outliers].head())

sorted_outliers_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_outliers[:outlier_nums], "Unnamed: 0"]
})
print(len(sorted_outliers_df))
save_csv_file(sorted_outliers_df, f"{output_dir}/mlp/data/outliers.csv")


# Near-duplicate issues
duplicate_results = lab.get_issues("near_duplicate")
sorted_duplicates = duplicate_results.sort_values("near_duplicate_score").index
print("Near-duplicate issues")
display(duplicate_results.sort_values("near_duplicate_score").head())
sorted_duplicates_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_duplicates, "Unnamed: 0"]
})
print(len(sorted_duplicates_df))
save_csv_file(sorted_duplicates_df[:duplicates_nums], f"{output_dir}/mlp/data/duplicates.csv")


121184
Label Issues


,Unnamed: 0,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,...,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC,given_label,predicted_label
56543,338796,Dublin 15,Mid-terrace house,2019,Final,A3,54.41,105.28,0.18,0.12,...,268006038,1.868260e-02,0.018683,NaN,NaN,NaN,0.360,0.399,2,1
106655,342190,Dublin 15,Mid-terrace house,2019,Final,A3,54.44,105.04,0.18,0.12,...,268006038,1.868260e-02,0.018683,NaN,NaN,NaN,0.360,0.399,2,1
11116,781412,Dublin 10,Mid-terrace house,1940,Existing,D1,246.61,66.74,2.10,2.30,...,268057005,5.562300e-07,0.002500,NaN,NaN,NaN,NaN,NaN,9,14
40824,946507,Co. Wexford,Mid-terrace house,1981,Existing,F,405.45,89.58,0.95,0.16,...,247045025,5.562300e-07,0.002500,NaN,NaN,NaN,NaN,NaN,13,9
114825,761719,Co. Mayo,Detached house,1900,Existing,A1,-4.06,84.54,0.36,0.28,...,157106003,5.000000e-02,0.050000,1.032,0.0,11020.6,-0.015,-0.023,0,6


Outliers


,Unnamed: 0,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,...,ThirdWallAgeBandId,ThirdWallTypeId,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
99966,912339,Co. Galway,Detached house,1905,Existing,G,842.25,88.56,1.99,1.44,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,654970,Co. Louth,Detached house,2013,Provisional,A2,48.08,984.23,0.16,0.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.335,0.360
54221,1137153,Dublin 18,Detached house,1944,Existing,E1,311.70,510.65,2.10,2.30,...,NaN,NaN,267090003,0.050000,0.050000,0.0,155887.0,0.0,1.834,1.794
94225,331,Co. Kildare,Detached house,1943,Existing,G,481.80,71.00,2.16,0.13,...,NaN,NaN,87068001,0.089325,0.089325,NaN,NaN,NaN,NaN,NaN
3329,128742,Co. Laois,Mid-terrace house,1960,Existing,G,989.35,138.82,2.10,2.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2296
Near-duplicate issues


,is_near_duplicate_issue,near_duplicate_score,near_duplicate_sets,distance_to_nearest_neighbor
12099,True,0.0,"[110211, 12058, 46803]",0.0
67498,True,0.0,[104454],0.0
61290,True,0.0,"[31822, 41350, 44527, 90228, 113967, 58970]",0.0
80049,True,0.0,[30037],0.0
80048,True,0.0,"[38004, 60227, 21344, 68517]",0.0


121184


**Before Removal**

In [39]:
results = []

for i in range(n_splits):
    clf = get_mlp()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"])
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/mlp/data/results_before_removal.csv")

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 1, Accuracy: 0.630936171968478, F1 Score: 0.6099032437785947


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 2, Accuracy: 0.6258612864628461, F1 Score: 0.5953812419495647


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 3, Accuracy: 0.6358047613153444, F1 Score: 0.6001595410154521


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 4, Accuracy: 0.6324627635433429, F1 Score: 0.598625976708746


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 5, Accuracy: 0.6291467238818287, F1 Score: 0.5808560530241175


**After Removal**

In [40]:
for i in range(n_splits):
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    label_issue_index = pd.read_csv(f"{output_dir}/mlp/data/label_issues.csv")
    train_df_removed = train_df[~train_df["Unnamed: 0"].isin(label_issue_index["Unnamed: 0"])]
    print(len(train_df_removed), len(train_df_removed)/len(train_df))
    save_csv_file(train_df_removed, f"{output_dir}/split_{i+1}/train_removed.csv")

/tmp/ipykernel_33/2653796351.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


67374 0.6949570383817962


/tmp/ipykernel_33/2653796351.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


67595 0.6972366344497509


/tmp/ipykernel_33/2653796351.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


67148 0.6926258677421684


/tmp/ipykernel_33/2653796351.py:2: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


67460 0.6958441210145749


/tmp/ipykernel_33/2653796351.py:2: DtypeWarning: Columns (156,162) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")


67403 0.69524899946363


In [41]:
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir {config_dir} \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/train_removed.csv" \
      --output_csv_name "processed_train_removed" \
    """
    os.system(command)

In [42]:
results = []

for i in range(n_splits):
    clf = get_mlp()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train_removed.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"])
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/mlp/data/results_after_removal.csv")

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 1, Accuracy: 0.6438503115071997, F1 Score: 0.5925289080138606


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 2, Accuracy: 0.6495853447208813, F1 Score: 0.6060533149750091


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 3, Accuracy: 0.6501217147336716, F1 Score: 0.6039937686841632


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 4, Accuracy: 0.6477699385237446, F1 Score: 0.6114444649265209


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Split: 5, Accuracy: 0.6499422346921935, F1 Score: 0.599780301129709


**Scarf Embedding**

In [43]:

X_processed = pd.DataFrame(all_emb, columns=[f"feature_{i+1}" for i in range(all_emb.shape[1])])
labels = processed['EnergyRating']
clf = get_mlp()
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=kf,
    method="predict_proba",
)
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_processed.values)

knn_graph = KNN.kneighbors_graph(mode="distance")
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)
display(lab.report())

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py

Finding label issues ...
Finding outlier issues ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Finding class_imbalance issues ...
Finding underperforming_group issues ...

Audit complete. 60573 issues found in the dataset.
Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
         label       48826
near_duplicate        7915
       outlier        3832

Dataset Information: num_examples: 121184, num_classes: 15


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 48826
Overall dataset quality in terms of this issue: 0.5702

Examples representing most severe instances of this issue:
       is_label_issue   label_score  given_label  predicted_label
24816            True  5.667480e-07            0                9
974

None

In [44]:
# Label Issues
label_issues_num =  48890
outlier_nums =  3752
duplicates_nums = 7983

issue_results = lab.get_issues("label")
sorted_issues = issue_results.sort_values("label_score").index

sorted_issues_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_issues, "Unnamed: 0"]
})
save_csv_file(sorted_issues_df[:label_issues_num], f"{output_dir}/mlp/emb/label_issues.csv")
print(len(sorted_issues_df))

print("Label Issues")
display(X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head())

# Outliers
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index
print("Outliers")
display(X_raw.iloc[sorted_outliers].head())

sorted_outliers_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_outliers[:outlier_nums], "Unnamed: 0"]
})
print(len(sorted_outliers_df))
save_csv_file(sorted_outliers_df, f"{output_dir}/mlp/emb/outliers.csv")


# Near-duplicate issues
duplicate_results = lab.get_issues("near_duplicate")
sorted_duplicates = duplicate_results.sort_values("near_duplicate_score").index
print("Near-duplicate issues")
display(duplicate_results.sort_values("near_duplicate_score").head())
sorted_duplicates_df = pd.DataFrame({
    "Unnamed: 0": X_raw.loc[sorted_duplicates, "Unnamed: 0"]
})
print(len(sorted_duplicates_df))
save_csv_file(sorted_duplicates_df[:duplicates_nums], f"{output_dir}/mlp/emb/duplicates.csv")


121184
Label Issues


,Unnamed: 0,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,...,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC,given_label,predicted_label
24816,752940,Co. Galway,Semi-detached house,2001,Existing,A1,-21.61,65.64,0.55,0.13,...,067057011,5.000000e-02,0.050000,1.143,12712.3,15487.50,0.507,-0.108,0,9
97402,1191541,Dublin 2,Mid-floor apartment,1993,Existing,A3,73.17,65.98,0.60,0.00,...,268140012,2.500000e-03,0.002500,NaN,NaN,NaN,NaN,NaN,2,11
95888,216971,Dublin 9,Ground-floor apartment,1983,Existing,G,504.80,37.16,0.60,0.00,...,268070008,6.929249e-04,0.012347,NaN,NaN,NaN,NaN,NaN,14,6
75141,544955,Dublin 15,Semi-detached house,1977,Existing,B1,89.09,96.48,0.51,0.19,...,267028025,5.562300e-07,0.002500,0.377,13075.0,5209.68,0.648,0.594,3,9
49470,260377,Co. Cork,Detached house,1997,Existing,D2,294.93,152.70,0.55,0.40,...,47227003,5.000000e-02,0.050000,NaN,NaN,NaN,NaN,NaN,10,5


Outliers


,Unnamed: 0,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,...,ThirdWallAgeBandId,ThirdWallTypeId,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
99966,912339,Co. Galway,Detached house,1905,Existing,G,842.25,88.56,1.99,1.44,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,654970,Co. Louth,Detached house,2013,Provisional,A2,48.08,984.23,0.16,0.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.335,0.360
110292,227162,Co. Wicklow,Detached house,1970,Existing,D2,262.02,860.31,2.05,2.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94225,331,Co. Kildare,Detached house,1943,Existing,G,481.80,71.00,2.16,0.13,...,NaN,NaN,87068001,0.089325,0.089325,NaN,NaN,NaN,NaN,NaN
54221,1137153,Dublin 18,Detached house,1944,Existing,E1,311.70,510.65,2.10,2.30,...,NaN,NaN,267090003,0.050000,0.050000,0.0,155887.0,0.0,1.834,1.794


3752
Near-duplicate issues


,is_near_duplicate_issue,near_duplicate_score,near_duplicate_sets,distance_to_nearest_neighbor
33156,True,0.0,[37576],0.0
104541,True,0.0,[56916],0.0
54683,True,0.0,"[74172, 67755]",0.0
25554,True,0.0,[85878],0.0
104553,True,0.0,"[11315, 53432]",0.0


121184


**Before Removal**

In [ ]:
results = []

for i in range(n_splits):
    clf = get_mlp()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}/train.npy")
    test_X = np.load(f"{output_dir}/split_{i+1}/test.npy")
    clf.fit(train_X, train_df["EnergyRating"])
    acc = clf.score(test_X, test_df["EnergyRating"],
             )
    predict = clf.predict(test_X)
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/mlp/emb/results_before_removal.csv")

**After Removal**

In [ ]:
for i in range(n_splits):
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}/train.npy")
    
    label_issue_index = pd.read_csv(f"{output_dir}/mlp/emb/label_issues.csv")
    train_df_removed = train_df[~train_df["Unnamed: 0"].isin(label_issue_index["Unnamed: 0"])]
    train_X_removed = train_X[train_df_removed.index]
    train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].str.strip()
    train_df_removed['EnergyRating'] = train_df_removed['EnergyRating'].map(original_rating_encoding)
    print(len(train_df_removed), len(train_df_removed)/len(train_df))
    save_csv_file(train_df_removed, f"{output_dir}/split_{i+1}/train_removed.csv")
    np.save(f"{output_dir}/split_{i+1}/train_removed.npy", train_X_removed)

In [ ]:
results = []

for i in range(n_splits):
    clf = get_mlp()
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/train_removed.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    train_X = np.load(f"{output_dir}/split_{i+1}//train_removed.npy")
    test_X = np.load(f"{output_dir}/split_{i+1}/test.npy")
    clf.fit(train_X, train_df["EnergyRating"])
    acc = clf.score(test_X, test_df["EnergyRating"],
             )
    predict = clf.predict(test_X)
    f1 = f1_score(test_df["EnergyRating"], predict, average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")
    results.append({"Split": i+1, "Accuracy": acc, "F1 Score": f1})
results_df = pd.DataFrame(results)
save_csv_file(results_df, f"{output_dir}/mlp/emb/results_after_removal.csv")



In [ ]:
import matplotlib.pyplot as plt
def compare_results(data_before, data_after, emb_before, emb_after, title):
    fig, ax = plt.subplots(2,1, figsize=(8, 8), sharex=True)
    fig.suptitle(title)

    # Plot Accuracy comparison
    bar_width = 0.2
    splits = data_before['Split']

    ax[0].bar(splits - 1.5 * bar_width, data_before['Accuracy'], width=bar_width, label='Data Before Removal')
    ax[0].bar(splits - 0.5 * bar_width, data_after['Accuracy'], width=bar_width, label='Data After Removal')
    ax[0].bar(splits + 0.5 * bar_width, emb_before['Accuracy'], width=bar_width, label='Emb Before Removal')
    ax[0].bar(splits + 1.5 * bar_width, emb_after['Accuracy'], width=bar_width, label='Emb After Removal')
    ax[0].set_title('Accuracy Comparison')
    ax[0].set_ylabel('Accuracy')

    # Plot F1 Score comparison
    ax[1].bar(splits - 1.5 * bar_width, data_before['F1 Score'], width=bar_width, label='Data Before Removal')
    ax[1].bar(splits - 0.5 * bar_width, data_after['F1 Score'], width=bar_width, label='Data After Removal')
    ax[1].bar(splits + 0.5 * bar_width, emb_before['F1 Score'], width=bar_width, label='Emb Before Removal')
    ax[1].bar(splits + 1.5 * bar_width, emb_after['F1 Score'], width=bar_width, label='Emb After Removal')
    ax[1].set_title('F1 Score Comparison')
    ax[1].set_xlabel('Split')
    ax[1].set_ylabel('F1 Score')

    # plt.subplots_adjust(right=0.8)

    # Create a single legend outside of the plot, on the right
    handles, labels = ax[0].get_legend_handles_labels()
    # fig.legend(handles, labels)
    fig.legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5))

    # Display the plot
    plt.tight_layout(rect=[0, 0, 0.85, 0.95])
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data_before = pd.read_csv(f"{output_dir}/random_forest/data/results_before_removal.csv")
data_after = pd.read_csv(f"{output_dir}/random_forest/data/results_after_removal.csv")
emb_before = pd.read_csv(f"{output_dir}/random_forest/emb/results_before_removal.csv")
emb_after = pd.read_csv(f"{output_dir}/random_forest/emb/results_after_removal.csv")

compare_results(data_before, data_after, emb_before, emb_after, "Random Forest")

data_before = pd.read_csv(f"{output_dir}/mlp/data/results_before_removal.csv")
data_after = pd.read_csv(f"{output_dir}/mlp/data/results_after_removal.csv")
emb_before = pd.read_csv(f"{output_dir}/mlp/emb/results_before_removal.csv")
emb_after = pd.read_csv(f"{output_dir}/mlp/emb/results_after_removal.csv")

compare_results(data_before, data_after, emb_before, emb_after, "MLP")


